In [ ]:
import pandas as pd
import numpy as np
import nltk
#nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
import os
import sys
# !pip install contractions
# !pip uninstall torchtext
# !pip install torchtext==0.10.0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#amazon_reviews_data=pd.read_csv(os.path.join(sys.path[0], 'amazon_reviews_us_Jewelry_v1_00.tsv'),sep='\t',on_bad_lines='skip')
# stored in /content/gdrive/mydrive/colab notebooks in google colab
amazon_reviews_data=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/amazon_reviews_us_Jewelry_v1_00.tsv',sep='\t',on_bad_lines='skip')
amazon_reviews_data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,0.0,0.0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,0.0,0.0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,0.0,0.0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31
3,US,5350721,R2I150CX5IVY9Q,B0006SW2WU,569859289,Surgical Stainless Steel Domed 9mm Fishbone Ri...,Jewelry,5,0.0,0.0,N,Y,Five Stars,Love it. Fits great. Super comfortable and nea...,2015-08-31
4,US,24484424,R1RM9ICOOA9MQ3,B009YPDW70,332947422,"Sterling Silver Family Pendant Necklace, 18""",Jewelry,5,0.0,0.0,N,Y,... a Mother's Day gift for my Mom and she lov...,Got this as a Mother's Day gift for my Mom and...,2015-08-31


In [ ]:
ratings_reviews_data=amazon_reviews_data[['star_rating','review_body']]
ratings_reviews_data.head()

,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


**We select 20000 reviews randomly from each rating class to create a balanced dataset**

In [ ]:
print ("Unique Ratings before cleaning\n")
print (ratings_reviews_data['star_rating'].unique())
print ("\n")
pd.options.mode.chained_assignment = None
# Above statement shows that there are some strings which have same value as int and some values like nan, date in star rating columns which need to be dropped


ratings_reviews_data['star_rating'] = pd.to_numeric(ratings_reviews_data['star_rating'], errors='coerce').fillna(0).astype(np.int64) # converting all values to int 

ratings_reviews_data = ratings_reviews_data[ratings_reviews_data['star_rating'].notna()] # keeping only those ratings which are not nan
ratings_reviews_data = ratings_reviews_data[ratings_reviews_data['review_body'].notna()] # keeping only those reviews which are not nan
#ratings_reviews_data = ratings_reviews_data[ratings_reviews_data['review_headline'].notna()] # keeping only those reviews which are not nan

print ("Unique Ratings after cleaning:\n")
print (ratings_reviews_data['star_rating'].unique())

# We cannot simply remove all nan since there maybe nans in only date columns which we will drop later

print ("\nCount of number of values for each rating\n")
print (ratings_reviews_data['star_rating'].value_counts())

#n = 20000
n=5000

#ratings_sampled_data=ratings_reviews_data.groupby('star_rating').apply(lambda x: x.sample(min(n,len(x)))).reset_index(drop=True) # this line of code should be used for chosing random samples


ratings_sampled_data=ratings_reviews_data.groupby('star_rating').head(10000).reset_index(drop=True) # to be consistent with the python file use this line of code 
# In the above statement I have selected 12000 from each class since the session was crashing when selecting all 20000. I have indiividually tried for 20000 points and results improve than current results



print ("\n Final Count of number of values for each rating keeping 20k per each rating\n")
print (ratings_sampled_data['star_rating'].value_counts())

# y=ratings_sampled_data['star_rating']
# X=ratings_sampled_data.drop('star_rating',axis=1)

#  from sklearn.model_selection import train_test_split
#  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)  # split the data

print (ratings_sampled_data)

Unique Ratings before cleaning

[5 1 4 3 2 nan '4' '5' '2' '3' '1' '2012-12-21']


Unique Ratings after cleaning:

[5 1 4 3 2]

Count of number of values for each rating

5    1080871
4     270424
3     159654
1     155002
2     100797
Name: star_rating, dtype: int64

 Final Count of number of values for each rating keeping 20k per each rating

5    10000
1    10000
4    10000
3    10000
2    10000
Name: star_rating, dtype: int64
       star_rating                                        review_body
0                5  so beautiful even tho clearly not high end ......
1                5  Great product.. I got this set for my mother, ...
2                5  Exactly as pictured and my daughter's friend l...
3                5  Love it. Fits great. Super comfortable and nea...
4                5  Got this as a Mother's Day gift for my Mom and...
...            ...                                                ...
49995            2  Poor quality. Get what you pay for in this cas...
49996 

**Data Cleaning**

In [ ]:
import contractions

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')

def txt_cleaning(text):
    text = contractions.fix(text)
    text = text.lower() # converting all reviews to lowercase
    text = BeautifulSoup(text, "lxml").text # removing HTML tags and URL's
    text = text.replace(r's*https?://S+(s+|$)', ' ').strip()
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text

ratings_sampled_data['review_body'] = ratings_sampled_data['review_body'].apply(txt_cleaning)
ratings_sampled_data['review_body']=  ratings_sampled_data['review_body'].replace({' +':' '},regex=True)
ratings_sampled_data

,star_rating,review_body
0,5,so beautiful even though clearly not high end ...
1,5,great product i got this set for my mother as ...
2,5,exactly as pictured and my daughters friend lo...
3,5,love it fits great super comfortable and neat ...
4,5,got this as a mothers day gift for my mom and ...
...,...,...
49995,2,poor quality get what you pay for in this case...
49996,2,this perfume confuses me so much i really want...
49997,2,way to big
49998,2,the necklace and very fragile and broke fast t...


**Only basic data cleaning has been performed such as removing tags, links or other bad charcaters. I am not removing stop words.**

**We need to split the data into train and test first in order to avoid leakage**

In [ ]:
from sklearn.model_selection import train_test_split

X = ratings_sampled_data.iloc[:, 1].values
y = ratings_sampled_data.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

**Word Embedding**

**Loading the google new pretrained embeddings**

In [ ]:
# import gensim.downloader as api
# model = api.load('word2vec-google-news-300')

**Saving the pretrained embeddings in order to save the time to reload them again**

In [ ]:
# model.init_sims(replace=True)

# model.save("/content/gdrive/MyDrive/Colab Notebooks/stored_pretrained_google_model.model")

**Checking the semantic similarities for the pretrained embeddings**

In [ ]:
from gensim.models import KeyedVectors
from threading import Semaphore
model = KeyedVectors.load('/content/gdrive/MyDrive/Colab Notebooks/stored_pretrained_google_model.model', mmap='r')
#model.syn0norm = model.syn0  # prevent recalc of normed vectors
#Semaphore(0).acquire()  # just hang until process killed
#model.most_similar(positive=['woman','king'], negative=['man'], topn = 5)

In [ ]:
model.most_similar(positive=['purple','blue'], negative=['pink'], topn = 5)

[('maroon', 0.6170071363449097),
 ('burgundy', 0.5899689793586731),
 ('red', 0.5830079913139343),
 ('crimson', 0.5826750993728638),
 ('yellow_plaid_pants', 0.5572718381881714)]

In [ ]:
model.most_similar(positive=['daughter','husband'], negative=['sister'], topn = 5)

[('wife', 0.7284117937088013),
 ('son', 0.7059542536735535),
 ('mother', 0.6460014581680298),
 ('father', 0.6453131437301636),
 ('fiance', 0.6444408893585205)]

In [ ]:
model.most_similar(positive=['ring','locket'], negative=['finger'], topn = 5)

[('necklace', 0.5243216753005981),
 ('rings', 0.5141297578811646),
 ('pendant', 0.4835904836654663),
 ('locket_necklace', 0.45760393142700195),
 ('jade_bracelet', 0.45525795221328735)]

**Looking at the first semantic similarity, we are feeding 3 colors so naturally the most similar choice should be a color. Similarly in the second example, wife is most similar to husband since husband and wife occur together mostly in general. In the third example, ring and finger are related so locket and necklace are also related.**

**Training Word2Vec model using our dataset**

In [ ]:
X_tr=pd.DataFrame(columns=['review_body'])
y_tr=pd.DataFrame(columns=['star_rating'])
X_ts=pd.DataFrame(columns=['review_body'])
y_ts=pd.DataFrame(columns=['star_rating'])
X_tr['review_body']=X_train
X_ts['review_body']=X_test
y_tr['star_rating']=y_train
y_ts['star_rating']=y_test
X_tr

,review_body
0,completely fake
1,very happy with it
2,it is bigger than i was expecting it almost to...
3,color change after wearing twice
4,bought this and after about a while it started...
...,...
47995,i wore this chain three times and then it brok...
47996,its too big
47997,feel apart right after she put it on my mom lo...
47998,looks cute but tickles my nose too bad to wear


In [ ]:
from gensim.models import Word2Vec
training_corpus = X_tr.review_body.apply(lambda x: x.split(" "))  # splitting all the words into tokens since word2vec expects list of strings
w2v_model= Word2Vec(sentences=training_corpus, size=300, window=11, min_count=10)

In [ ]:
w2v_model.wv.most_similar(positive=['daughter','husband'], negative=['sister'], topn = 5)

[('fianc', 0.784647524356842),
 ('son', 0.7840499877929688),
 ('hubby', 0.7766162157058716),
 ('boyfriend', 0.7722760438919067),
 ('wife', 0.7680268883705139)]

In [ ]:
w2v_model.wv.most_similar(positive=['purple','blue'], negative=['pink'], topn = 5)

[('sapphire', 0.8536776900291443),
 ('topaz', 0.8481096029281616),
 ('deep', 0.8479543924331665),
 ('amethyst', 0.8239973187446594),
 ('red', 0.8147281408309937)]

In [ ]:
w2v_model.wv.most_similar(positive=['ring','locket'], negative=['finger'], topn = 5)

[('necklace', 0.6700347661972046),
 ('pin', 0.6254616379737854),
 ('pendant', 0.6015681028366089),
 ('item', 0.5900734663009644),
 ('cross', 0.5855146050453186)]

**Pretrained word embeddings seem to show more similar results due to huge vocabulary. Word2vec might sometimes also not find a word in the vocabulary. But in case of our 3 examples both are perfroming well. There might be case when the dataset does not have enough words which is when word2vec might not give good similarities. On the other hand sometimes word2vec might perform better if there are words in the dataset which need to go together. For ex, in a certain data husband and son might be more related than husband and wife so in this case word2vec might perform better**

In [ ]:
# !pip install torchtext==0.9.1
import torchtext.legacy
from torchtext.legacy import data
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

tokenizer = lambda s: s.split() # tokenize the reviews

Text = data.Field(tokenize=tokenizer, batch_first=True, include_lengths=True)
Label = data.Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None)

fields = [('star_rating', Label),('review_body', Text)]

In [ ]:
# We need to convert the pandas dataframe to torchtext dataset

list_of_examples = [data.Example.fromlist(row.tolist(),fields=fields) for _, row in ratings_sampled_data.iterrows()]  #converting each row of dataframe to torch text example which cintains text and label fields

torch_data = data.Dataset(examples=list_of_examples, fields=fields)  #constructing the torchtext dataset

In [ ]:
import random
training_data, testing_data = torch_data.split(split_ratio=0.8, random_state=random.seed(15))

In [ ]:
train_data, validation_data = training_data.split(split_ratio=0.8, random_state=random.seed(15))

**Building the vocabulary**

**Loading an embedding layer with pretrained google news and bulding the vocabulary using pytorch**

In [ ]:
import torch
from torchtext import vocab
# Text.build_vocab(train_data, validation_data) 
# word2index = {token: token_index for token_index, token in enumerate(model.index2word)}
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Text.vocab.set_vectors(word2index, torch.from_numpy(model.vectors).float().to(device), model.vector_size)
# Label.build_vocab(train_data)

**In all the 3 cells below, I am checking if the word from our data is present in the google pretrained vectors or not. If yes, then I am appending else it will be encoded with zeros of dimension 300 since the dimension of each word is 300. Then we calculate the average of all the appended words. This will be done for training data, validation data and testing data.**

In [ ]:
x_training=[]
y_training=[]
for i in range(len(training_data)):
  doc=[]
  for word in vars(training_data[i])['review_body']:
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
  if (len(doc)!=0):
    x_training.append(np.mean(doc, axis=0))
    y_training.append(vars(training_data[i])['star_rating'])
  else:
    x_training.append(np.zeros(300))
    y_training.append(vars(training_data[i])['star_rating'])

X_training_simple_avg = np.array(x_training)
y_training_simple_avg = np.array(y_training)

In [ ]:
x_tr=[]
y_tr=[]
x_ts=[]
y_ts=[]
for i in range(len(train_data)):
  doc=[]
  for word in vars(train_data[i])['review_body']:
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
  if (len(doc)!=0):
    x_tr.append(np.mean(doc, axis=0))
    y_tr.append(vars(train_data[i])['star_rating'])
  else:
    x_tr.append(np.zeros(300))
    y_tr.append(vars(train_data[i])['star_rating'])

In [ ]:
x_val=[]
y_val=[]
for i in range(len(validation_data)):
  doc=[]
  for word in vars(validation_data[i])['review_body']:
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
  if (len(doc)!=0):
    x_val.append(np.mean(doc, axis=0))
    y_val.append(vars(validation_data[i])['star_rating'])
  else:
    x_val.append(np.zeros(300))
    y_val.append(vars(validation_data[i])['star_rating'])

In [ ]:
for i in range(len(testing_data)):
  doc=[]
  for word in vars(testing_data[i])['review_body']:
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
  if (len(doc)!=0):
    x_ts.append(np.mean(doc, axis=0))
    y_ts.append(vars(testing_data[i])['star_rating'])
  else:
    x_ts.append(np.zeros(300))
    y_ts.append(vars(testing_data[i])['star_rating'])

**Calculating results using SVM and average vector as features**

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

X_tr_simple_avg = np.array(x_tr)
X_val_simple_avg = np.array(x_val)
X_ts_simple_avg = np.array(x_ts)
y_tr_simple_avg = np.array(y_tr)
y_val_simple_avg = np.array(y_val)
y_ts_simple_avg = np.array(y_ts)

from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]} 

#svm_model = GridSearchCV(LinearSVC(), param_grid, refit = True, verbose = 0, scoring='f1_micro')

svm_model=LinearSVC() 

svm_model.fit(X_training_simple_avg, y_training_simple_avg)
y_pred_simple = svm_model.predict(X_ts_simple_avg)

print (classification_report(y_ts_simple_avg, y_pred_simple))
print('Accuracy: {:.2f}'.format(accuracy_score(y_ts_simple_avg, y_pred_simple)))

              precision    recall  f1-score   support

           1       0.51      0.72      0.59      2361
           2       0.40      0.26      0.32      2391
           3       0.41      0.39      0.40      2451
           4       0.42      0.32      0.36      2377
           5       0.60      0.75      0.67      2420

    accuracy                           0.49     12000
   macro avg       0.47      0.49      0.47     12000
weighted avg       0.47      0.49      0.47     12000

Accuracy: 0.49


Results for SVM taking TFIDF (copied from HW1 directly as per the post https://piazza.com/class/l7102doc7aa3ob/post/223)

           1       0.57      0.71      0.64      4029
           2       0.44      0.35      0.39      3977
           3       0.45      0.40      0.42      3981
           4       0.49      0.43      0.46      3993
           5       0.65      0.78      0.70      4020

    accuracy                           0.53     20000


**From above we can say that using TF-IDF has given us better results for SVM than using average vectors by 4 percent. This might be because when we are using average as a feature, there might some information loss as to not giving more weight to important words in the corpus. Also, the average will just be 1 300 dim vector containing weight of all the words in the review. But the average might vary depending on the word present in the corpus. If the word is less important and is being used in average, it might have an impact on the embedding values. In TFIDF, however since it depends on the frequency, model might be peprforming better. However, there might be cases when average might perform better because we are using pre trained embeddings which are based on context. Accuracy for this SVM can also be improved by hyperparameter tuning.**

**Simple Perceptron using average as a feature**

In [ ]:
from sklearn.metrics import classification_report

from sklearn.linear_model import Perceptron

per_model=Perceptron()

per_model.fit(X_training_simple_avg, y_training_simple_avg)
y_pred_simple = per_model.predict(X_ts_simple_avg)

print (classification_report(y_ts_simple_avg, y_pred_simple))
print('Accuracy: {:.2f}'.format(accuracy_score(y_ts_simple_avg, y_pred_simple)))

              precision    recall  f1-score   support

           1       0.54      0.55      0.54      2361
           2       0.28      0.70      0.40      2391
           3       0.61      0.02      0.03      2451
           4       0.44      0.10      0.17      2377
           5       0.60      0.75      0.66      2420

    accuracy                           0.42     12000
   macro avg       0.49      0.42      0.36     12000
weighted avg       0.49      0.42      0.36     12000

Accuracy: 0.42


**Simple perceptron using TFIDF (copied from HW1 directly as per post https://piazza.com/class/l7102doc7aa3ob/post/223)**


           1       0.65      0.29      0.40      4029
           2       0.30      0.20      0.24      3977
           3       0.26      0.68      0.38      3981
           4       0.38      0.18      0.24      3993
           5       0.61      0.49      0.55      4020

    accuracy                           0.37     20000

**From above, we can say that for perceptron, average as a feature is performing better than TFIDF. As mentioned above there might be cases when average might perform better because we are using pre trained embeddings which are based on context and perceptron might be better able to identify the context because of pretrained embeddings over TF-IDF. It also depends on the model and hyperparameter tuning.**

**Feedforward Neural Networks by taking the average of word vectors for a review**

In [ ]:
import torch
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.nn import Sigmoid
from numpy import vstack
class avgMLP(torch.nn.Module):
    def __init__(self, input_size):
        super(avgMLP, self).__init__()
        self.dropout = torch.nn.Dropout(0.5)
        self.hidden_size1=50
        self.hidden_size2=10
        self.output_dim=5  # since we have 5 class classification problem
        self.hidden1 = torch.nn.Linear(input_size, self.hidden_size1)
        self.act1 = torch.nn.ReLU()
        self.hidden2 = torch.nn.Linear(self.hidden_size1, self.hidden_size2)
        self.act2 = torch.nn.ReLU()
        self.hidden3 = torch.nn.Linear(self.hidden_size2, self.output_dim)
        self.act3 = Sigmoid()
        
    def forward(self, X):
        X = X.view(X.size(0), -1) 
        X = self.hidden1(X)
        X = self.act1(X)
        #X = self.dropout(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        #X = self.dropout(X)
        X = self.hidden3(X)
        return X

In [ ]:
from torch.nn import CrossEntropyLoss
torch.manual_seed(0)
from torch.optim import SGD
from numpy import vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


def accuracy(probs, target):
  winners = probs.argmax(dim=1)+1  # getting the class with max probability and adding 1 since pytorch has indices 0 to n
  target=target+1
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy


def train_model(train_dl, y_tr_simple, model):
  epoch_loss = 0
  epoch_acc = 0
  criterion = CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  optimizer.zero_grad()
  output = model(train_dl)
  target=torch.from_numpy(y_tr_simple)
  target=target-1
  loss = criterion(output, target)
  acc = accuracy(output, target)
  loss.backward()
  optimizer.step()
  epoch_loss += loss.item()
  epoch_acc += acc.item()
  return epoch_loss, epoch_acc


def evaluate_model(test_dl, y_ts, model):
   epoch_loss = 0
   epoch_acc = 0
   criterion = CrossEntropyLoss()
   with torch.no_grad():
        yhat = model(test_dl)
        target = torch.from_numpy(y_ts)
        target=target-1
        loss = criterion(yhat, target)
        acc = accuracy(yhat, target)   
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        return epoch_loss, epoch_acc 


avgMLP_model = avgMLP(300)
X_train = torch.from_numpy(X_tr_simple_avg)
X_val = torch.from_numpy(X_val_simple_avg)
X_test = torch.from_numpy(X_ts_simple_avg)
X_train = X_train.float()
X_val=X_val.float()
X_test=X_test.float()
best_valid_loss = float('inf')
for epoch in range(1000):
   train_loss, train_acc = train_model(X_train, y_tr_simple_avg, avgMLP_model)
   valid_loss, valid_acc = evaluate_model(X_val, y_val_simple_avg, avgMLP_model)
   if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(avgMLP_model.state_dict(), 'saved_weights'+'_'+'avgMLP_model'+'.pt')
   print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
   print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')



	Train Loss: 1.632 | Train Acc: 19.98%
	 Val. Loss: 1.626 |  Val. Acc: 19.54%
	Train Loss: 1.625 | Train Acc: 19.98%
	 Val. Loss: 1.619 |  Val. Acc: 19.54%
	Train Loss: 1.617 | Train Acc: 19.98%
	 Val. Loss: 1.610 |  Val. Acc: 19.54%
	Train Loss: 1.609 | Train Acc: 19.98%
	 Val. Loss: 1.599 |  Val. Acc: 21.99%
	Train Loss: 1.597 | Train Acc: 22.80%
	 Val. Loss: 1.584 |  Val. Acc: 25.33%
	Train Loss: 1.582 | Train Acc: 25.90%
	 Val. Loss: 1.566 |  Val. Acc: 28.35%
	Train Loss: 1.563 | Train Acc: 28.96%
	 Val. Loss: 1.547 |  Val. Acc: 28.64%
	Train Loss: 1.544 | Train Acc: 29.05%
	 Val. Loss: 1.533 |  Val. Acc: 29.98%
	Train Loss: 1.530 | Train Acc: 30.02%
	 Val. Loss: 1.526 |  Val. Acc: 30.13%
	Train Loss: 1.522 | Train Acc: 30.27%
	 Val. Loss: 1.518 |  Val. Acc: 26.96%
	Train Loss: 1.514 | Train Acc: 26.69%
	 Val. Loss: 1.514 |  Val. Acc: 30.31%
	Train Loss: 1.509 | Train Acc: 30.44%
	 Val. Loss: 1.504 |  Val. Acc: 28.36%
	Train Loss: 1.500 | Train Acc: 28.06%
	 Val. Loss: 1.504 |  Val

In [ ]:
avgMLP_model.load_state_dict(torch.load(os.path.join("saved_weights_avgMLP_model.pt")))
test_loss, test_acc = evaluate_model(X_test, y_ts_simple_avg, avgMLP_model)
print(f' For MLP Feedforwrd taking avg of vectors: Test Acc: {test_acc * 100:.2f}%')

 For MLP Feedforwrd taking avg of vectors: Test Acc: 44.80%


**Feedforward Neural Networks  by concatenating the first 10 word vectors for each review**

In [ ]:
x_tr=[]
y_tr=[]
x_ts=[]
y_ts=[]
for i in range(len(train_data)):
  count=0
  doc=[]
  for word in vars(train_data[i])['review_body']:
    if count>=10:
      break
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
    count+=1
  for k in range(count,10):
    doc.append(np.zeros(300))
  x_tr.append(doc)
  y_tr.append(vars(train_data[i])['star_rating'])

In [ ]:
x_val=[]
y_val=[]
for i in range(len(validation_data)):
  count=0
  doc=[]
  for word in vars(validation_data[i])['review_body']:
    if count>=10:
      break
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
    count+=1
  for k in range(count,10):
    doc.append(np.zeros(300))
  x_val.append(doc)
  y_val.append(vars(validation_data[i])['star_rating'])

In [ ]:
for i in range(len(testing_data)):
  count=0
  doc=[]
  for word in vars(testing_data[i])['review_body']:
    if count>=10:
      break
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
    count+=1
  for k in range(count,10):
    doc.append(np.zeros(300))
  x_ts.append(doc)
  y_ts.append(vars(testing_data[i])['star_rating'])

X_tr_simple_concat = np.array(x_tr)
X_val_simple_concat = np.array(x_val)
X_ts_simple_concat = np.array(x_ts)
y_tr_simple_concat = np.array(y_tr)
y_val_simple_concat = np.array(y_val)
y_ts_simple_concat = np.array(y_ts)

In [ ]:
import torch
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.nn import Sigmoid
from numpy import vstack
class concatMLP(torch.nn.Module):
    def __init__(self, input_size):
        super(concatMLP, self).__init__()
        self.dropout = torch.nn.Dropout(0.5)
        self.hidden_size1=50
        self.hidden_size2=10
        self.output_dim=5
        self.hidden1 = torch.nn.Linear(input_size, self.hidden_size1)
        #kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = torch.nn.ReLU()
        # second hidden layer
        self.hidden2 = torch.nn.Linear(self.hidden_size1, self.hidden_size2)
        #kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = torch.nn.ReLU()
        # third hidden layer and output
        self.hidden3 = torch.nn.Linear(self.hidden_size2, self.output_dim)
        #xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
 
        
    def forward(self, X):
        X = X.view(X.size(0), -1) 
        X = self.hidden1(X)
        X = self.act1(X)
        #X = self.dropout(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        #X = self.dropout(X)
        # third hidden layer and output
        X = self.hidden3(X)
        #X = self.act3(X)
        return X

In [ ]:
from torch.nn import CrossEntropyLoss
torch.manual_seed(0)
from torch.optim import SGD
from numpy import vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


def accuracy(probs, target):
  winners = probs.argmax(dim=1)+1  # getting the class with max probability and adding 1 since pytorch has indices 0 to n
  target=target+1
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy


def train_model(train_dl, y_tr_simple, model):
  epoch_loss = 0
  epoch_acc = 0
  criterion = CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  optimizer.zero_grad()
  output = model(train_dl)
  target=torch.from_numpy(y_tr_simple)
  target=target-1
  loss = criterion(output, target)
  acc = accuracy(output, target)
  loss.backward()
  optimizer.step()
  epoch_loss += loss.item()
  epoch_acc += acc.item()
  return epoch_loss, epoch_acc


def evaluate_model(test_dl, y_ts, model):
   epoch_loss = 0
   epoch_acc = 0
   criterion = CrossEntropyLoss()
   with torch.no_grad():
        yhat = model(test_dl)
        target = torch.from_numpy(y_ts)
        target=target-1
        loss = criterion(yhat, target)
        acc = accuracy(yhat, target)   
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        return epoch_loss, epoch_acc 


concatMLP_model = concatMLP(3000)
X_train = torch.from_numpy(X_tr_simple_concat)
X_val = torch.from_numpy(X_val_simple_concat)
X_test = torch.from_numpy(X_ts_simple_concat)
X_train = X_train.float()
X_val=X_val.float()
X_test=X_test.float()
best_valid_loss = float('inf')
for epoch in range(500):
   train_loss, train_acc = train_model(X_train, y_tr_simple_concat, concatMLP_model)
   valid_loss, valid_acc = evaluate_model(X_val, y_val_simple_concat, concatMLP_model)
   if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(concatMLP_model.state_dict(), 'saved_weights'+'_'+'concatMLP_model'+'.pt')
   print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
   print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')



	Train Loss: 1.618 | Train Acc: 19.98%
	 Val. Loss: 1.609 |  Val. Acc: 20.64%
	Train Loss: 1.607 | Train Acc: 21.44%
	 Val. Loss: 1.592 |  Val. Acc: 21.96%
	Train Loss: 1.591 | Train Acc: 22.25%
	 Val. Loss: 1.569 |  Val. Acc: 25.92%
	Train Loss: 1.566 | Train Acc: 27.00%
	 Val. Loss: 1.582 |  Val. Acc: 30.06%
	Train Loss: 1.582 | Train Acc: 30.46%
	 Val. Loss: 1.549 |  Val. Acc: 31.05%
	Train Loss: 1.546 | Train Acc: 31.46%
	 Val. Loss: 1.568 |  Val. Acc: 28.59%
	Train Loss: 1.565 | Train Acc: 28.84%
	 Val. Loss: 1.522 |  Val. Acc: 32.51%
	Train Loss: 1.517 | Train Acc: 33.06%
	 Val. Loss: 1.538 |  Val. Acc: 30.02%
	Train Loss: 1.533 | Train Acc: 30.43%
	 Val. Loss: 1.501 |  Val. Acc: 33.68%
	Train Loss: 1.493 | Train Acc: 34.75%
	 Val. Loss: 1.515 |  Val. Acc: 32.37%
	Train Loss: 1.509 | Train Acc: 32.83%
	 Val. Loss: 1.481 |  Val. Acc: 34.83%
	Train Loss: 1.470 | Train Acc: 36.23%
	 Val. Loss: 1.498 |  Val. Acc: 32.91%
	Train Loss: 1.489 | Train Acc: 33.93%
	 Val. Loss: 1.468 |  Val

In [ ]:
concatMLP_model.load_state_dict(torch.load(os.path.join("saved_weights_concatMLP_model.pt")))
test_loss, test_acc = evaluate_model(X_test, y_ts_simple_concat, concatMLP_model)
print(f' For MLP Feedforward taking concatenation of vectors: Test Acc: {test_acc * 100:.2f}%')

 For MLP Feedforward taking concatenation of vectors: Test Acc: 42.41%


**From above we can conclude that, for feedforward, average as a feature is performing better than word concatenation. This is becasue we are only concatenating the first 10 word vectors. So there is a  possibility that more important information or context is present after 10 words and we are missing that information. For ex, I bought this ring for my mother on her birthday as a gift and I really loved the ring and would definitely buy again; beautiful color, amazing quality etc. In this ex, all the positive context is present after 10 words so we might classify this as neutral but is actually positive. But for avg, we have given weight to all the words. Hence, avg might be performing better. Also w.r.t models SVM is performing better than feedforward by a margin but feedforward is performing better than simple perceptron. Also, we can further improve feedforward by adding more epochs and parameter tuning to cross SVM**

**Recurrent Neural Networks  from scratch without using nn.rnn**

In [ ]:
x_tr=[]
y_tr=[]
x_ts=[]
y_ts=[]
for i in range(len(train_data)):
  count=0
  doc=[]
  for word in vars(train_data[i])['review_body']:
    if count>=20:
      break
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
    count+=1
  for k in range(count,20):
    doc.append(np.zeros(300))
  x_tr.append(doc)
  y_tr.append(vars(train_data[i])['star_rating'])


In [ ]:
x_val=[]
y_val=[]
for i in range(len(validation_data)):
  count=0
  doc=[]
  for word in vars(validation_data[i])['review_body']:
    if count>=20:
      break
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
    count+=1
  for k in range(count,20):
    doc.append(np.zeros(300))
  x_val.append(doc)
  y_val.append(vars(validation_data[i])['star_rating'])


In [ ]:
for i in range(len(testing_data)):
  count=0
  doc=[]
  for word in vars(testing_data[i])['review_body']:
    if count>=20:
      break
    if word in model.vocab:
      doc.append(model[word])
    else:
      doc.append(np.zeros(300))
    count+=1
  for k in range(count,20):
    doc.append(np.zeros(300))
  x_ts.append(doc)
  y_ts.append(vars(testing_data[i])['star_rating'])

In [ ]:
X_tr_simple_RNN = np.array(x_tr)


In [ ]:
X_val_simple_RNN = np.array(x_val)

In [ ]:
X_ts_simple_RNN = np.array(x_ts)

In [ ]:
y_tr_simple_RNN = np.array(y_tr)

In [ ]:
y_val_simple_RNN = np.array(y_val)
y_ts_simple_RNN = np.array(y_ts)

In [ ]:
import torch
#weights = torch.FloatTensor(Text.vocab.vectors)
class RNN(torch.nn.Module):
    def __init__(self,input_size):
        super(RNN,self).__init__()
        self.hidden_dim=20
        self.n_layers=1
        self.i2h = torch.nn.Linear(input_size + self.hidden_dim, self.hidden_dim)
        self.i2o = torch.nn.Linear(input_size + self.hidden_dim, 5)
        self.softmax = torch.nn.LogSoftmax(dim=1)
        #self.rnn = torch.nn.RNN(input_size, self.hidden_dim, self.n_layers, batch_first=True, nonlinearity="relu")
        #self.linear  = torch.nn.Linear(self.hidden_dim, 5)   # each word represented by 300 dimensions 
   
    def init_hidden(self,batch_size):
      return torch.zeros(1, self.hidden_dim)
      # hidden=torch.zeros(self.n_layers,batch_size,self.hidden_dim)
      # return hidden
   
   
    def forward(self, X, hidden): 
        X = X.view(1, -1)
        combined = torch.cat((X, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
        
        # X = X.view(1, -1)
        # X = X.unsqueeze(0)
        # hidden=self.init_hidden(1) 
        # X,hidden = self.rnn(X,hidden)
        # X = X.view(X.size(0), self.hidden_dim)
        # X = self.linear(X)
        # return X,hidden

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from numpy import vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


def accuracy(probs, target):
  winners = probs.argmax(dim=1)+1  # getting the class with max probability and adding 1 since pytorch has indices 0 to n
  target=target+1
  if winners==target:
    return 1
  else:
    return 0
 
def train_model(train_dl, y_tr_simple, model):
  epoch_loss = 0
  epoch_acc = 0
  #criterion = CrossEntropyLoss()
  criterion = torch.nn.NLLLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
  optimizer.zero_grad()
  hidden = model.init_hidden(1)
  for vector in train_dl:  # taking one word at a time
    output,hidden = model(vector,hidden)
  y_tr_simple=y_tr_simple-1
  loss = criterion(output, torch.tensor([y_tr_simple]))
  acc = accuracy(output, torch.tensor([y_tr_simple]))
  loss.backward()
  optimizer.step()
  
  return loss, acc


def evaluate_model(test_dl, y_ts, model):
   epoch_loss = 0
   epoch_acc = 0
   criterion = CrossEntropyLoss()
   hidden = model.init_hidden(1)
   for test_vector in test_dl:  # passing word by word
          yhat,hidden = model(test_vector,hidden)
   y_ts=y_ts- 1
   loss = criterion(yhat, torch.tensor([y_ts]))
   acc = accuracy(yhat, torch.tensor([y_ts]))

   return loss,acc

rnn_model = RNN(300)
X_train = torch.from_numpy(X_tr_simple_RNN)
X_val = torch.from_numpy(X_val_simple_RNN)
testing_data = torch.from_numpy(X_ts_simple_RNN)
X_train = X_train.float()
X_val=X_val.float()
X_test=testing_data.float()
count=0
best_valid_loss = float('inf')
for epoch in range(10):
   count+=1
   tr_loss=0
   tr_acc=0
   val_loss=0
   val_acc=0 
   for i, x in enumerate(X_train):
      train_loss, train_acc = train_model(x, y_tr_simple_RNN[i], rnn_model)
      tr_loss+=train_loss
      tr_acc+=train_acc
   for j, y in enumerate(X_val):
      valid_loss, valid_acc = evaluate_model(y, y_val_simple_RNN[j], rnn_model)
      val_loss+=valid_loss
      val_acc+=valid_acc
   if val_loss < best_valid_loss:
              best_valid_loss = val_loss
              torch.save(rnn_model.state_dict(), 'saved_weights'+'_'+'RNN'+'.pt')
   print ("Epoch: "+str(count))
   print(f'\tTrain Loss: {(train_loss * 100)/len(X_train):.3f} | Train Acc: {(tr_acc * 100)/len(X_train):.2f}%')
   print(f'\t Val. Loss: {(valid_loss * 100)/len(X_val):.3f} |  Val. Acc: {(val_acc * 100)/len(X_val):.2f}%')
   print ("-----------------------------------")


Epoch: 1
	Train Loss: 0.003 | Train Acc: 21.05%
	 Val. Loss: 0.019 |  Val. Acc: 23.05%
-----------------------------------
Epoch: 2
	Train Loss: 0.009 | Train Acc: 22.96%
	 Val. Loss: 0.038 |  Val. Acc: 22.21%
-----------------------------------
Epoch: 3
	Train Loss: 0.000 | Train Acc: 24.81%
	 Val. Loss: 0.087 |  Val. Acc: 22.68%
-----------------------------------
Epoch: 4
	Train Loss: 0.004 | Train Acc: 27.85%
	 Val. Loss: 0.072 |  Val. Acc: 23.60%
-----------------------------------
Epoch: 5
	Train Loss: 0.000 | Train Acc: 25.91%
	 Val. Loss: 0.165 |  Val. Acc: 23.09%
-----------------------------------
Epoch: 6
	Train Loss: 0.000 | Train Acc: 28.23%
	 Val. Loss: 0.226 |  Val. Acc: 22.16%
-----------------------------------
Epoch: 7
	Train Loss: 0.002 | Train Acc: 27.95%
	 Val. Loss: 0.158 |  Val. Acc: 22.89%
-----------------------------------
Epoch: 8
	Train Loss: 0.000 | Train Acc: 26.96%
	 Val. Loss: 0.207 |  Val. Acc: 23.36%
-----------------------------------
Epoch: 9
	Train 

**Note: I ran my code on only 10 epochs due to session crash and infrastructure issues. I have also experimented sending entire sequence to RNN at the end of the notebook. Results are decent with whole sequence.**

In [ ]:
rnn_model.load_state_dict(torch.load(os.path.join("saved_weights_RNN.pt")))
t_loss=0
t_acc=0
for j, t in enumerate(X_test):
      test_loss, test_acc = evaluate_model(t, y_ts_simple_RNN[j], rnn_model)
      t_loss+=test_loss
      t_acc+=test_acc
print(f' For RNN: Test Acc: {(t_acc * 100)/len(X_test):.2f}%')

 For RNN: Test Acc: 22.99%


**Based on above, we can say that feedforward models are giving much better results than RNN. One reason for this can be because of zero padding. If there are multiple reviews with length  < 20 which we are doing for RNN, this can lead to a lot of zero vectors being fed. This can lead to a poor performance**

**Since, we are not being judged on accuracy, I have not performed significant tuning. Also, the data is small due to session crash. Results will definitely get better with more data.**

**Gated Recurrent Unit Cell**

In [ ]:
import torch
#weights = torch.FloatTensor(Text.vocab.vectors)
class GRNN(torch.nn.Module):
    def __init__(self,input_size):
        super(GRNN,self).__init__()
        self.hidden_dim=20
        self.n_layers=1

        # self.i2h = torch.nn.Linear(input_size + self.hidden_dim, self.hidden_dim)
        # self.i2o = torch.nn.Linear(input_size + self.hidden_dim, 5)
        # self.softmax = torch.nn.LogSoftmax(dim=1)

        self.grnn = torch.nn.GRU(input_size, self.hidden_dim, self.n_layers, batch_first=True)
        self.linear  = torch.nn.Linear(self.hidden_dim, 5)   # each word represented by 300 dimensions 
        self.relu = torch.nn.ReLU()
        #self.softmax = torch.nn.LogSoftmax(dim=1)
   
    def init_hidden(self,batch_size):
      #hidden=torch.zeros(self.n_layers,batch_size,self.hidden_dim).requires_grad_()
      #return hidden
      return torch.zeros(1, 1, self.hidden_dim)
   
   
    def forward(self, X, hidden): 
        X = X.view(1, -1)
        X = X.unsqueeze(0)
        hidden=self.init_hidden(1) 
        X,hidden = self.grnn(X,hidden)
        X = X.view(X.size(0), self.hidden_dim)
        X = self.linear(self.relu(X))
        #X = self.softmax(X)
        return X,hidden
        # print (hidden.shape)
        # combined = torch.cat((X, hidden),1)
        # hidden = self.i2h(combined)
        # output = self.i2o(combined)
        # output = self.softmax(output)
        # return output, hidden

In [24]:
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from numpy import vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


def accuracy(probs, target):
  winners = probs.argmax(dim=1)+1  # getting the class with max probability and adding 1 since pytorch has indices 0 to n
  target=target+1
  if winners==target:
    return 1
  else:
    return 0
  # corrects = (winners == target)
  # accuracy = corrects.sum().float()
  # return accuracy


def train_model(train_dl, y_tr_simple, model):
  epoch_loss = 0
  epoch_acc = 0
  criterion = CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  optimizer.zero_grad()
  hidden = model.init_hidden(1)
  for vector in train_dl:  # taking one word at a time
    output,hidden = model(vector,hidden)
  y_tr_simple=y_tr_simple-1
  loss = criterion(output, torch.tensor([y_tr_simple]))
  acc = accuracy(output, torch.tensor([y_tr_simple]))
  loss.backward()
  optimizer.step()
  #epoch_loss += loss.item()
  #epoch_acc += acc.item()
  #return epoch_loss, epoch_acc
  return loss, acc


def evaluate_model(test_dl, y_ts, model):
   epoch_loss = 0
   epoch_acc = 0
   criterion = CrossEntropyLoss()
   hidden = model.init_hidden(1)
   for test_vector in test_dl:
          yhat,hidden = model(test_vector,hidden)
   y_ts=y_ts- 1
   loss = criterion(yhat, torch.tensor([y_ts]))
   acc = accuracy(yhat, torch.tensor([y_ts]))
   #epoch_loss += loss.item()
   #epoch_acc += acc.item()
   #return epoch_loss, epoch_acc 
   return loss,acc


# grnn_model = GRNN(6000)
grnn_model = GRNN(300)
X_train = torch.from_numpy(X_tr_simple_RNN)
X_val = torch.from_numpy(X_val_simple_RNN)
testing_data = torch.from_numpy(X_ts_simple_RNN)
X_train = X_train.float()
X_val=X_val.float()
X_test=testing_data.float()
count=0
best_valid_loss = float('inf')
for epoch in range(10):
   count+=1
   tr_loss=0
   tr_acc=0
   val_loss=0
   val_acc=0 
   for i, x in enumerate(X_train):
      train_loss, train_acc = train_model(x, y_tr_simple_RNN[i], grnn_model)
      tr_loss+=train_loss
      tr_acc+=train_acc
   for j, y in enumerate(X_val):
      valid_loss, valid_acc = evaluate_model(y, y_val_simple_RNN[j], grnn_model)
      val_loss+=valid_loss
      val_acc+=valid_acc
   if val_loss < best_valid_loss:
              best_valid_loss = val_loss
              torch.save(grnn_model.state_dict(), 'saved_weights'+'_'+'GRNN'+'.pt')
   print ("Epoch: "+str(count))
   print(f'\tTrain Loss: {(train_loss * 100) /len(X_train):.3f} | Train Acc: {(tr_acc * 100)/len(X_train):.2f}%')
   print(f'\t Val. Loss: {(valid_loss * 100) /len(X_val):.3f} |  Val. Acc: {(val_acc * 100)/len(X_val):.2f}%')
   print ("-----------------------------------")


Epoch: 1
	Train Loss: 0.004 | Train Acc: 19.96%
	 Val. Loss: 0.026 |  Val. Acc: 20.10%
-----------------------------------
Epoch: 2
	Train Loss: 0.006 | Train Acc: 20.61%
	 Val. Loss: 0.019 |  Val. Acc: 20.27%
-----------------------------------
Epoch: 3
	Train Loss: 0.003 | Train Acc: 20.31%
	 Val. Loss: 0.042 |  Val. Acc: 20.12%
-----------------------------------
Epoch: 4
	Train Loss: 0.013 | Train Acc: 20.75%
	 Val. Loss: 0.063 |  Val. Acc: 19.86%
-----------------------------------
Epoch: 5
	Train Loss: 0.008 | Train Acc: 20.50%
	 Val. Loss: 0.072 |  Val. Acc: 19.90%
-----------------------------------
Epoch: 6
	Train Loss: 0.008 | Train Acc: 20.73%
	 Val. Loss: 0.029 |  Val. Acc: 20.34%
-----------------------------------
Epoch: 7
	Train Loss: 0.010 | Train Acc: 20.93%
	 Val. Loss: 0.035 |  Val. Acc: 19.81%
-----------------------------------
Epoch: 8
	Train Loss: 0.014 | Train Acc: 20.84%
	 Val. Loss: 0.095 |  Val. Acc: 19.88%
-----------------------------------
Epoch: 9
	Train 

In [25]:
grnn_model.load_state_dict(torch.load(os.path.join("saved_weights_GRNN.pt")))
t_loss=0
t_acc=0
for j, t in enumerate(X_test):
      test_loss, test_acc = evaluate_model(t, y_ts_simple_RNN[j], grnn_model)
      t_loss+=test_loss
      t_acc+=test_acc
print(f' For Gated RNN: Test Acc: {(t_acc * 100)/len(X_test):.2f}%')

 For Gated RNN: Test Acc: 20.14%


**Based on above, we can say that RNN performs better than gated by a small margin. This can be since RNN works well when the sequence is small and there might be cases when the reviews are short. Also, I haven't performed significant tuning due to insufficient computational power. Gated RNN might work well after fine tuning.**

References:
https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-rnn-for-text-classification-tasks#3

https://galhever.medium.com/sentiment-analysis-with-pytorch-part-4-lstm-bilstm-model-84447f6c4525

https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/

https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_recurrent_neuralnetwork/